## follow

In [1]:
import pandas as pd

In [2]:
# read finalTable.csv all columns as string
finalTable = pd.read_csv('finalTable.csv', dtype=str)
finalTable

,twitter_id,author_id,hashtag,frequency,full_name,state,region,party
0,233737858,142332083,RonsRoadTour,118,Ron Johnson,WI,north,Republican
1,10615232,118740781,iaSen,109,Chuck Grassley,IA,north,Republican
2,10615232,118740781,GrassleyWorks,104,Chuck Grassley,IA,north,Republican
3,131546062,131546062,Ian,92,Rick Scott,FL,south,Republican
4,88784440,2164909022,ILikeMike,80,Mike Lee,UT,north,Republican
...,...,...,...,...,...,...,...,...
2037,983348251972816896,983348251972816896,desperation,1,Cindy Hyde-Smith,MS,south,Republican
2038,983348251972816896,983348251972816896,happydiwali,1,Cindy Hyde-Smith,MS,south,Republican
2039,983348251972816896,983348251972816896,shopsmall,1,Cindy Hyde-Smith,MS,south,Republican
2040,983348251972816896,983348251972816896,smallbusinesssaturday,1,Cindy Hyde-Smith,MS,south,Republican


In [3]:
# give me unique values of twitter_id and author_id
print(len(finalTable['twitter_id'].unique()))
print(len(finalTable['author_id'].unique()))

87
118


In [6]:
import sqlite3

# connect to SQLite database
conn = sqlite3.connect('twitterFollows.db')  
c = conn.cursor()

# create tables in the SQLite database
c.execute('''
    CREATE TABLE IF NOT EXISTS FollowData
    (User1 TEXT NOT NULL,
    User2 TEXT NOT NULL,
    FollowEachOther BOOLEAN NOT NULL);
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS UserFollowing
    (UserId TEXT NOT NULL,
    FollowingId TEXT NOT NULL);
''')



In [7]:
import tweepy
import pandas as pd

def get_following(id, client):
    following_ids = []
    for page in tweepy.Paginator(client.get_users_following, id, max_results=1000):
        following_ids.extend([user['id'] for user in page.data])
    return following_ids

user_ids = finalTable['author_id'][0:2].tolist()  # Replace with your list of user ids

print(f"Found {len(user_ids)} users to process.")

# Replace with your Bearer token
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAGYKkQEAAAAAbNiwJ4AF4PTE8km%2FFTVoB7ewbds%3Dv7YaQmbfxG4r6NnkRvvqfeysnZ0ylCf3EIOHHiKzMRxomN8w4l", wait_on_rate_limit=True)

user_following = {user_id: get_following(user_id, client) for user_id in user_ids}
        
for user_id in user_ids:
    print(f"Fetching following ids for user {user_id}.")
    following = get_following(user_id, client)
    user_following[user_id] = following
    print(f"User {user_id} is following {len(following)} users.")
    # Insert the following ids into the UserFollowing table
    for following_id in following:
        c.execute("INSERT INTO UserFollowing (UserId, FollowingId) VALUES (?, ?)", 
                  (user_id, following_id))
        

print("Finished fetching following ids for all users.")
        
for i in range(len(user_ids)):
    for j in range(i+1, len(user_ids)):
        print(f"Checking if users {user_ids[i]} and {user_ids[j]} follow each other.")
        result = user_ids[j] in user_following[user_ids[i]] and user_ids[i] in user_following[user_ids[j]]
        if result:
            print(f"Users {user_ids[i]} and {user_ids[j]} follow each other.")
        else:
            print(f"Users {user_ids[i]} and {user_ids[j]} do not follow each other.")
        c.execute("INSERT INTO FollowData (User1, User2, FollowEachOther) VALUES (?, ?, ?)", 
                  (user_ids[i], user_ids[j], result))

        
print("Finished processing all users.")

# commit your changes and close the connection
conn.commit()
conn.close()

Found 2 users to process.
Fetching following ids for user 142332083.
User 142332083 is following 2709 users.
Fetching following ids for user 118740781.


Rate limit exceeded. Sleeping for 794 seconds.


User 118740781 is following 1715 users.
Finished fetching following ids for all users.
Checking if users 142332083 and 118740781 follow each other.
Users 142332083 and 118740781 do not follow each other.
Finished processing all users.


In [ ]:
import tweepy
import pandas as pd

def get_following(id, client):
    following_ids = []
    for page in tweepy.Paginator(client.get_users_following, id, max_results=1000):
        following_ids.extend([user['id'] for user in page.data])
    return following_ids

# Replace with your Bearer token
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAGYKkQEAAAAAbNiwJ4AF4PTE8km%2FFTVoB7ewbds%3Dv7YaQmbfxG4r6NnkRvvqfeysnZ0ylCf3EIOHHiKzMRxomN8w4l", wait_on_rate_limit=True)

user_ids = finalTable['author_id'].tolist()  # Replace with your list of user ids

user_following = {user_id: get_following(user_id, client) for user_id in user_ids}

results = []
for i in range(len(user_ids)):
    for j in range(i+1, len(user_ids)):
        result = user_ids[j] in user_following[user_ids[i]] and user_ids[i] in user_following[user_ids[j]]
        results.append([user_ids[i], user_ids[j], result])

df = pd.DataFrame(results, columns=['User1', 'User2', 'FollowEachOther'])

print(df)

## mention

In [1]:
import sqlite3
conn = sqlite3.connect("D://Dropbox//projeler//ndsu_polarization//twitterVeriToplama//twitter004.db")
c = conn.cursor()
c

### create mentioned accounts table (once)

In [44]:
# create new table
c.execute('''
    CREATE TABLE mentionedUsers3 (
        author_id TEXT,
        twitter_id TEXT,
        mentioned_id TEXT,
        state TEXT,
        party TEXT
    )
''')

In [45]:
# query data from finalTweetsTable
c.execute('SELECT author_id, twitter_id, state, party, entities FROM finalTweetsTable')
rows = c.fetchall()

In [47]:
rows[33]

('45645232',
 '224285242',
 'CO',
 'Democrat',
 '{"urls": [{"start": 66, "end": 89, "url": "https://t.co/uLu9rpj9MV", "expanded_url": "https://secure.actblue.com/donate/mfb-soc-2022?refcode=tw-221031", "display_url": "secure.actblue.com/donate/mfb-soc…", "images": [{"url": "https://pbs.twimg.com/news_img/1587194490879504384/JtRfySe_?format=jpg&name=orig", "width": 1200, "height": 657}, {"url": "https://pbs.twimg.com/news_img/1587194490879504384/JtRfySe_?format=jpg&name=150x150", "width": 150, "height": 150}], "status": 200, "title": "Chip in to support Michael Bennet", "description": "Make a donation now to help Michael fight off the GOP and protect the Democratic majority.", "unwound_url": "https://secure.actblue.com/donate/mfb-soc-2022?refcode=tw-221031"}]}')

In [ ]:
# parse JSON and insert data into new table
import json
for row in rows:
    author_id, twitter_id, state, party, entities = row
    if entities is not None:
        try:
            entities_json = json.loads(entities)
            for mention in entities_json.get('mentions', []):
                mentioned_id = mention.get('id')
                c.execute('INSERT INTO MentionedUsers3 (author_id, twitter_id, state, party, mentioned_id) VALUES (?, ?, ?, ?, ?)', (author_id, twitter_id, state, party, mentioned_id))
        except json.JSONDecodeError:
            print(f"Error parsing JSON for author_id {author_id}: {entities}")

# commit the transactions
conn.commit()

### create matrix

In [2]:
# read * from mentionedUsers table to df
import pandas as pd
mentionedUsers = pd.read_sql_query("SELECT * FROM mentionedUsers3", conn)
mentionedUsers
# burada id sayısı çok fazla çünkü tüm mentionlananlar var. biz sadece kendi listemizdekileri alacağız.

,author_id,twitter_id,mentioned_id,state,party
0,45645232,224285242,4750779374,CO,Democrat
1,45645232,224285242,483010877,CO,Democrat
2,45645232,224285242,117839957,CO,Democrat
3,45645232,224285242,540352897,CO,Democrat
4,45645232,224285242,228591530,CO,Democrat
...,...,...,...,...,...
10392,1352287997853622273,1352287997853622273,123935925,GA,Democrat
10393,1352287997853622273,1352287997853622273,14275261,GA,Democrat
10394,1352287997853622273,1352287997853622273,957024441338417154,GA,Democrat
10395,1352287997853622273,1352287997853622273,44783853,GA,Democrat


In [3]:
print(len(mentionedUsers["mentioned_id"].unique()))
print(len(mentionedUsers["author_id"].unique()))
print(len(mentionedUsers["twitter_id"].unique()))


3479
121
90


In [4]:
# what are the number of unique elements in author_id different than twitter_id?
print(len(mentionedUsers["author_id"].unique()) - len(mentionedUsers["twitter_id"].unique()))

31


In [5]:
# get unique values from 'author_id' column
unique_user1 = mentionedUsers['author_id'].unique()

# filter 'mentioned_id' column based on these unique values
filtered_data = mentionedUsers[mentionedUsers['mentioned_id'].isin(unique_user1)]

In [6]:
print(len(filtered_data))
# buradaki sayı finalTable'daki sayıdan az ama normal. herkes birilerini mentionlamıyor.
print(len(filtered_data['author_id'].unique()))
print(len(filtered_data['twitter_id'].unique()))
print(len(filtered_data['mentioned_id'].unique()))


1805
108
88
113


In [7]:
filtered_data

,author_id,twitter_id,mentioned_id,state,party
13,45645232,224285242,45645232,CO,Democrat
38,45645232,224285242,82489457,CO,Democrat
41,102477372,278124059,102477372,CT,Democrat
43,102477372,278124059,102477372,CT,Democrat
45,102477372,278124059,102477372,CT,Democrat
...,...,...,...,...,...
10314,1356714265894408196,1356714265894408196,1356714265894408196,GA,Democrat
10320,1356714265894408196,1356714265894408196,1356714265894408196,GA,Democrat
10354,1356714265894408196,1356714265894408196,1356714265894408196,GA,Democrat
10360,1356714265894408196,1356714265894408196,1048784496,GA,Democrat


In [8]:
# kendini mentionlayanları çıkar
filtered_data = filtered_data[filtered_data['author_id'] != filtered_data['mentioned_id']]
filtered_data

,author_id,twitter_id,mentioned_id,state,party
38,45645232,224285242,82489457,CO,Democrat
67,102477372,278124059,131546062,CT,Democrat
72,102477372,278124059,131546062,CT,Democrat
112,3006258669,5558312,5558312,AR,Republican
113,3006258669,5558312,382791093,AR,Republican
...,...,...,...,...,...
10212,1351236267707654145,1351236267707654145,476256944,CA,Democrat
10260,1351236267707654145,1351236267707654145,13218102,CA,Democrat
10282,1351236267707654145,1351236267707654145,1048784496,CA,Democrat
10360,1356714265894408196,1356714265894408196,1048784496,GA,Democrat


In [9]:
# show duplicated rows
filtered_data[filtered_data.duplicated()]

,author_id,twitter_id,mentioned_id,state,party
72,102477372,278124059,131546062,CT,Democrat
116,3006258669,5558312,5558312,AR,Republican
120,3006258669,5558312,5558312,AR,Republican
125,3006258669,5558312,5558312,AR,Republican
126,3006258669,5558312,5558312,AR,Republican
...,...,...,...,...,...
10191,1346873610875654147,1346873610875654147,968650362,TN,Republican
10194,1346873610875654147,1346873610875654147,109071031,TN,Republican
10199,1346873610875654147,1346873610875654147,968650362,TN,Republican
10212,1351236267707654145,1351236267707654145,476256944,CA,Democrat


In [10]:
filtered_data = filtered_data.drop_duplicates()
filtered_data

,author_id,twitter_id,mentioned_id,state,party
38,45645232,224285242,82489457,CO,Democrat
67,102477372,278124059,131546062,CT,Democrat
112,3006258669,5558312,5558312,AR,Republican
113,3006258669,5558312,382791093,AR,Republican
171,3006258669,5558312,76456274,AR,Republican
...,...,...,...,...,...
10206,1351236267707654145,1351236267707654145,1221242033530195970,CA,Democrat
10208,1351236267707654145,1351236267707654145,476256944,CA,Democrat
10260,1351236267707654145,1351236267707654145,13218102,CA,Democrat
10360,1356714265894408196,1356714265894408196,1048784496,GA,Democrat


In [11]:
len(filtered_data['author_id'].unique())

106

In [12]:
len(filtered_data['mentioned_id'].unique())

108

In [13]:
filtered_data

,author_id,twitter_id,mentioned_id,state,party
38,45645232,224285242,82489457,CO,Democrat
67,102477372,278124059,131546062,CT,Democrat
112,3006258669,5558312,5558312,AR,Republican
113,3006258669,5558312,382791093,AR,Republican
171,3006258669,5558312,76456274,AR,Republican
...,...,...,...,...,...
10206,1351236267707654145,1351236267707654145,1221242033530195970,CA,Democrat
10208,1351236267707654145,1351236267707654145,476256944,CA,Democrat
10260,1351236267707654145,1351236267707654145,13218102,CA,Democrat
10360,1356714265894408196,1356714265894408196,1048784496,GA,Democrat


In [22]:
# import pandas as pd

# # Assuming df is your original DataFrame
# df = pd.DataFrame({
#     'author_id': [45645232, 82489457, 3006258669, 3006258669, 3006258669],
#     'mentioned_id': [82489457, 45645232, 5558312, 382791093, 76456274]
# })

# # Create a new DataFrame with the desired structure
# new_df = pd.DataFrame({
#     'user1': df['author_id'],
#     'user2': df['mentioned_id'],
# })

# # Function to check if users mentioned each other
# def check_mention(row):
#     return any((df['author_id'] == row['user2']) & (df['mentioned_id'] == row['user1']))

# # Apply the function to each row
# new_df['mentions_each_other'] = new_df.apply(check_mention, axis=1)

# new_df['set'] = new_df.apply(lambda x: {x['user1'], x['user2']}, axis=1)

# new_df = new_df.drop_duplicates(subset=['set'])
# new_df


,user1,user2,mentions_each_other,set
0,45645232,82489457,True,"{45645232, 82489457}"
2,3006258669,5558312,False,"{5558312, 3006258669}"
3,3006258669,382791093,False,"{3006258669, 382791093}"
4,3006258669,76456274,False,"{76456274, 3006258669}"


In [23]:
import pandas as pd

# Assuming df is your original DataFrame
df = filtered_data

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame({
    'author_id': df['author_id'],
    'mentioned_id': df['mentioned_id'],
    'state': df['state'],
    'party': df['party'],
    'twitter_id': df['twitter_id']
})

# Function to check if users mentioned each other
def check_mention(row):
    return any((df['author_id'] == row['mentioned_id']) & (df['mentioned_id'] == row['author_id']))

# Apply the function to each row
new_df['mentions_each_other'] = new_df.apply(check_mention, axis=1)

new_df

,author_id,mentioned_id,state,party,twitter_id,mentions_each_other
38,45645232,82489457,CO,Democrat,224285242,True
67,102477372,131546062,CT,Democrat,278124059,False
112,3006258669,5558312,AR,Republican,5558312,True
113,3006258669,382791093,AR,Republican,5558312,True
171,3006258669,76456274,AR,Republican,5558312,False
...,...,...,...,...,...,...
10206,1351236267707654145,1221242033530195970,CA,Democrat,1351236267707654145,False
10208,1351236267707654145,476256944,CA,Democrat,1351236267707654145,True
10260,1351236267707654145,13218102,CA,Democrat,1351236267707654145,True
10360,1356714265894408196,1048784496,GA,Democrat,1356714265894408196,False


In [24]:
# create set from author_id and mentioned_id columns into new column   
new_df['set'] = new_df.apply(lambda x: {x['author_id'], x['mentioned_id']}, axis=1)
# drop duplicates in set column
new_df = new_df.drop_duplicates(subset=['set'])
new_df

,author_id,mentioned_id,state,party,twitter_id,mentions_each_other,set
38,45645232,82489457,CO,Democrat,224285242,True,"{45645232, 82489457}"
67,102477372,131546062,CT,Democrat,278124059,False,"{102477372, 131546062}"
112,3006258669,5558312,AR,Republican,5558312,True,"{3006258669, 5558312}"
113,3006258669,382791093,AR,Republican,5558312,True,"{3006258669, 382791093}"
171,3006258669,76456274,AR,Republican,5558312,False,"{3006258669, 76456274}"
...,...,...,...,...,...,...,...
10160,1346873610875654147,1017500185356853248,TN,Republican,1346873610875654147,False,"{1017500185356853248, 1346873610875654147}"
10185,1346873610875654147,968650362,TN,Republican,1346873610875654147,False,"{968650362, 1346873610875654147}"
10206,1351236267707654145,1221242033530195970,CA,Democrat,1351236267707654145,False,"{1221242033530195970, 1351236267707654145}"
10360,1356714265894408196,1048784496,GA,Democrat,1356714265894408196,False,"{1048784496, 1356714265894408196}"


In [26]:
new_df

,author_id,mentioned_id,state,party,twitter_id,mentions_each_other,set
38,45645232,82489457,CO,Democrat,224285242,1,"{45645232, 82489457}"
67,102477372,131546062,CT,Democrat,278124059,0,"{102477372, 131546062}"
112,3006258669,5558312,AR,Republican,5558312,1,"{3006258669, 5558312}"
113,3006258669,382791093,AR,Republican,5558312,1,"{3006258669, 382791093}"
171,3006258669,76456274,AR,Republican,5558312,0,"{3006258669, 76456274}"
...,...,...,...,...,...,...,...
10160,1346873610875654147,1017500185356853248,TN,Republican,1346873610875654147,0,"{1017500185356853248, 1346873610875654147}"
10185,1346873610875654147,968650362,TN,Republican,1346873610875654147,0,"{968650362, 1346873610875654147}"
10206,1351236267707654145,1221242033530195970,CA,Democrat,1351236267707654145,0,"{1221242033530195970, 1351236267707654145}"
10360,1356714265894408196,1048784496,GA,Democrat,1356714265894408196,0,"{1048784496, 1356714265894408196}"


In [ ]:
# delete author_id and mentioned_id columns
new_df = new_df.drop(['author_id', 'mentioned_id'], axis=1)

In [27]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is named df
df = new_df

# Replace the boolean values with integers
df['mentions_each_other'] = df['mentions_each_other'].astype(int)

# Create pivot tables for each direction of interaction
pivot_table1 = df.pivot(index='author_id', columns='mentioned_id', values='mentions_each_other').fillna(0)
pivot_table2 = df.pivot(index='mentioned_id', columns='author_id', values='mentions_each_other').fillna(0)

# Combine pivot tables
interaction_matrix = pivot_table1.add(pivot_table2, fill_value=0)

# Ensure the matrix is symmetric (i.e., user1->user2 = user2->user1)
interaction_matrix = interaction_matrix.clip(upper=1)

# Fill NaN values with 0
interaction_matrix = interaction_matrix.fillna(0)

# Show the interaction matrix
interaction_matrix

C:\Users\ysf\AppData\Local\Temp\ipykernel_23820\2444655614.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mentions_each_other'] = df['mentions_each_other'].astype(int)


,1017500185356853248,102477372,1048784496,1058520120,10615232,1068481578,1071402577,1074480192,1074518754,1078693601356509184,...,818554054309715969,82489457,88784440,899978622416695297,92186819,941000686275387392,946549322,968650362,970207298,983348251972816896
1017500185356853248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102477372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1048784496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1058520120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10615232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941000686275387392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
946549322,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
968650362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
970207298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
conn.close()